In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.set_cmap(plt.cm.Paired) # 色設定

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target

In [ ]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=1, 
                  train_size=0.8, 
                  test_size=0.2, 
                  random_state=0)

train_index, test_index = next(ss.split(X, y))

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')

In [ ]:
clf.C = 1
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf.C = 1000
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
import time

In [ ]:
C_range_exp = np.arange(-20.,20.)
C_range = 10 ** C_range_exp

scores = []
comp_time = []

for C in C_range:
    clf.C = C

    st = time.time()
    clf.fit(X_train, y_train)
    comp_time.append(time.time() - st)

    score = clf.score(X_test, y_test)
    print(C, score)
    scores.append(score)

scores = np.array(scores)
comp_time = np.array(comp_time)

In [ ]:
plt.plot(C_range_exp, scores)
plt.ylabel("accuracy")
plt.xlabel(r"$\log_{10}$(C)");
plt.show()

plt.plot(C_range_exp, comp_time)
plt.ylabel("computation time [sec]")
plt.xlabel(r"$\log_{10}$(C)");

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [ ]:
clf.C = 1
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf.C = 100
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
C_range_exp = np.arange(-20., 15.)
C_range = 10 ** C_range_exp

scores = []
comp_time = []

for C in C_range:
    clf.C = C

    st = time.time()
    clf.fit(X_train, y_train)
    comp_time.append(time.time() - st)

    score = clf.score(X_test, y_test)
    print(C, score)
    scores.append(score)

scores = np.array(scores)
comp_time = np.array(comp_time)

In [ ]:
plt.plot(C_range_exp, scores)
plt.ylabel("accuracy")
plt.xlabel(r"$\log_{10}$(C)");
plt.show()

plt.plot(C_range_exp, comp_time)
plt.ylabel("computation time [sec]")
plt.xlabel(r"$\log_{10}$(C)");

scikit-learn document: [Logistic regression](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

$$
\min_{w,b}
\frac{1}{2} \| w \|_2^2
+ C \sum_{i=1}^N
\log(\exp(-y_i(w^T x_i + b)) + 1)
$$

scikit-learn document: [SVC](http://scikit-learn.org/stable/modules/svm.html#svc)

$$
\min_{w, b, \{\xi_i\} }
\frac{1}{2} \| w \|_2^2
+ C \sum_{i=1}^N \xi_i
\quad \text{subject to}\quad 
y_i(w^T x_i + b) \ge 1 - \xi_i
$$


scikit-learn document: [Kernel functions](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)

$$
\min_{w, b, \{\xi_i\} }
\frac{1}{2} \| w \|_2^2
+ C \sum_{i=1}^N \xi_i
\quad \text{subject to}\quad 
y_i(w^T \phi(x_i) + b) \ge 1 - \xi_i
$$

$$
\text{rbf: }\quad
K(x, x')=
\phi(x)^T \phi(x')=
\exp(-\gamma \| x-x' \|^2)
$$

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200, # 20個生成
                  n_features=2, # 2次元
                  centers=2,    # クラスタ中心2個
                  cluster_std =2, # クラスタの大きさ（標準偏差）
                  random_state=3   # 乱数種（再現用）
                 )
plt.scatter(X[:, 0], X[:, 1], c=y, s=50); # 2次元散布図でプロット

In [ ]:

def plotSVMBoundary(X, clf, mesh=True, boundary=True):
    
    # if SVM, draw margine lines
    colors = ['k']*3
    linestyles = ['-']*3
    levels = [-1, 0, 1]
    # if SVM, plot support vecters
    plt.scatter(clf.support_vectors_[:, 0], 
                clf.support_vectors_[:, 1],
                s=80, facecolors='none')
    
    # plot range
    x_min = min(X[:,0])
    x_max = max(X[:,0])
    y_min = min(X[:,1])
    y_max = max(X[:,1])

    # visualizing decision function
    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j] # make a grid
    Z = clf.decision_function(np.c_[XX.ravel(), YY.ravel()]) # evaluate the value 
    Z = Z.reshape(XX.shape) # just reshape

    if mesh:
        plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired, zorder=-10)
    if boundary:
        plt.contour(XX, YY, Z, 
                    colors=colors,
                    linestyles=linestyles,
                    levels=levels) # draw lines (level=0:boundary, level=+-1:margine lines)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.kernel = 'rbf'
    
for C in [1e-10, 1e-5, 1, 1e5, 1e10, 1e20]:
   
    clf.C = C
    clf.fit(X,y)
    plotSVMBoundary(X, clf, mesh=True)
    plt.scatter(X[:, 0], X[:, 1], marker='o', s=50, c=y)
    plt.title("C={0:.0e}".format(C))
    
    plt.show()

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.kernel = 'rbf'
    
for C in 10 ** np.arange(-1.0, 6.0):
   
    clf.C = C
    clf.fit(X,y)
    plotSVMBoundary(X, clf, mesh=True)
    plt.scatter(X[:, 0], X[:, 1], marker='o', s=50, c=y)
    plt.title("C={0:.0e}".format(C))
    
    plt.show()

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200, # 20個生成
                  n_features=2, # 2次元
                  centers=[(-2, -2), (2, 2)],    # クラスタ中心2個
                  cluster_std=2, # クラスタの大きさ（標準偏差）
                  # random_state=3   # 乱数種（再現用）
                 )
plt.scatter(X[:, 0], X[:, 1], c=y, s=50); # 2次元散布図でプロット
plt.xlim(-10,10)
plt.ylim(-10,10)

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.kernel = 'rbf'

X_train, y_train = make_blobs(n_samples=200, # 20個生成
              n_features=2, # 2次元
              centers=[(-2, -2), (2, 2)],    # クラスタ中心2個
              cluster_std=2, # クラスタの大きさ（標準偏差）
              # random_state=3   # 乱数種（再現用）
             )

X_test, y_test = make_blobs(n_samples=200, # 20個生成
      n_features=2, # 2次元
      centers=[(-2, -2), (2, 2)],    # クラスタ中心2個
      cluster_std=2, # クラスタの大きさ（標準偏差）
      # random_state=3   # 乱数種（再現用）
     )


for C in 10 ** np.arange(-1.0, 6.0):
   
    clf.C = C
    
    plt.scatter(X_train[:, 0], X_train[:, 1], marker='o', s=50, c=y, label="training")

    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)

    plotSVMBoundary(X_train, clf, mesh=True)
    
    plt.scatter(X_test[:, 0], X_test[:, 1], marker='*', s=50, c=y, label="test")

    test_score = clf.score(X_test, y_test)
    
    plt.title("C={0:.0e}, train {1}, test {2}".format(C, train_score, test_score))

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
    plt.show()